In [5]:
import os
import pickle
import pandas as pd
import numpy as np
import tempfile
import tensorflow as tf
from sklearn.metrics import classification_report,confusion_matrix
model_dir = tempfile.mkdtemp(dir='/home/jiv/TF')
print(model_dir)
# config = tf.ConfigProto()
# config.gpu_options.allow_growth = True

ModuleNotFoundError: No module named 'tensorflow'

In [6]:
# Parameters of fast text are
size=350
window=11
min_n=2
max_n=11
epochs=80
embedding_matrix, x_train, y_train, x_test, y_test = BuildFasttext(size,window,min_n,max_n,epochs)
###########################################
from sklearn.model_selection import train_test_split
x_strat, x_dev, y_strat, y_dev = train_test_split(x_train, y_train,test_size=0.20,random_state=0,stratify=y_train)

model is saved earlier
Getting embedding matrix for vocubolory ...
File is already present
Getting features matrix for train and test data ...
File is already present


## Classification with Tensorflow

In [17]:
# Mapping words to there dictionary index and getting vector corroponding to input words
file_name=open("./data/compound_dic.pickle","rb")
my_dic = pickle.load(file_name)
train=pd.read_csv('./data/train.csv')
test=pd.read_csv('./data/test.csv')
x_train_temp=np.empty([len(train), 2], dtype=int)
x_test_voc=np.empty([len(test), 2], dtype=int)
for i in range(0,len(train)):
    x_train_temp[i]=[my_dic.get(train.iloc[i,1]),my_dic.get(train.iloc[i,2])]
for i in range(0,len(test)):
    x_test_voc[i]=[my_dic.get(test.iloc[i,1]),my_dic.get(test.iloc[i,2])]

from sklearn.model_selection import train_test_split
x_train_voc, x_dev_voc, y_train_voc, y_dev_voc = train_test_split(x_train_temp, y_train,test_size=0.20,random_state=0,stratify=y_train)

# Setting parameters for custom estimators
########################################################################
import tensorflow as tf
my_batch_size=100
my_num_epochs=75
my_steps=(my_num_epochs * x_train_voc.shape[0])/my_batch_size
# my_steps=2200
VOC_size=len(my_dic)
########################################################################
# Functions to pass the data to estimators
def my_train_fn(my_num_epochs):
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": np.array(x_train_voc)},
    y=np.array(y_train_voc.ravel()),
    num_epochs=my_num_epochs,
    shuffle=False)
    return train_input_fn

def my_dev_fn():
    test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": np.array(x_dev_voc)},
    y=np.array(y_dev_voc.ravel()),
    num_epochs=1,
    shuffle=False)
    return test_input_fn

def my_test_fn():
    test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": np.array(x_test_voc)},
    y=np.array(y_test.ravel()),
    num_epochs=1,
    shuffle=False)
    return test_input_fn

def my_initializer(shape=None, dtype=tf.float32, partition_info=None):
    assert dtype is tf.float32
    return embedding_matrix.astype(np.float32)

all_classifiers = {}
def train_and_eval(classifier,my_epochs,my_steps):
    all_classifiers[classifier.model_dir] = classifier
    classifier.train(input_fn=my_train_fn(my_epochs), steps= my_steps)
    eval_results = classifier.evaluate(input_fn=my_dev_fn(), steps= my_steps)
    print('Eval Accuracy is ')
    print(eval_results)
    dev_results = classifier.predict(input_fn=my_dev_fn())
    predictions=[]
    for pred in dev_results:
        predictions.append(pred)
    print(classification_report(y_dev_voc, pd.DataFrame(predictions)))
    print(confusion_matrix(y_dev_voc, pd.DataFrame(predictions)))
    
    print('################################################################')
    print(' ')
    print("Attention: This result is on test data")
    test_accu = classifier.evaluate(input_fn=my_test_fn(), steps= my_steps)
    print('Test Accuracy is ')
    print(test_accu)
    test_results = classifier.predict(input_fn=my_test_fn())
    test_predictions=[]
    for pred in test_results:
        test_predictions.append(pred)
    print(classification_report(y_test, pd.DataFrame(test_predictions)))
    print(confusion_matrix(y_test, pd.DataFrame(test_predictions)))


# Building feature matrix to inject estimator shape=(3,)
feature_x = tf.feature_column.numeric_column("x", shape=x_train_voc.shape)
# feature_x = tf.feature_column.numeric_column("x", shape=(x_train_voc.shape[1],))
feature_columns = [feature_x]

### 1) DNN based custom estimator

In [18]:
# my_steps
# my_steps=(x_train_voc.shape[0])/my_batch_size
# my_steps

In [19]:
params = {'embedding_initializer': my_initializer}
# my_steps=2000
# params['embedding_initializer']
def dnn_model_fn(
    features, # This is batch_features from input_fn
    labels,   # This is batch_labels from input_fn
    mode,params):    # And instance of tf.estimator.ModeKeys, see below
    input_layer = tf.contrib.layers.embed_sequence(ids=features["x"], embed_dim=size,vocab_size=VOC_size,
                                                   initializer=params['embedding_initializer'],
                                                   trainable=True)
    #######################################################################################
    training = mode == tf.estimator.ModeKeys.TRAIN
    dropout_emb = tf.layers.dropout(inputs=input_layer, rate=0.2, training=training)
#     pool = tf.reduce_max(input_tensor=input_layer, axis=1)
#     print(input_layer.shape.dims)
    flat = tf.contrib.layers.flatten(dropout_emb)
#     print(flat.shape.dims)

    h1 = tf.layers.dense(inputs=flat, units= 500, activation=tf.nn.relu)
#     h2 = tf.layers.dense(inputs=h1, units=400, activation=tf.nn.relu)
#     h3 = tf.layers.dense(inputs=h2, units=100, activation=tf.nn.relu)
#     print(h1.shape.dims)
#     h2 = tf.layers.dense(inputs=h1, units=100, activation=tf.nn.relu)
#     print(h2.shape.dims)
    logits = tf.layers.dense(inputs=h1, units=4)
    ######################################################################################
    # Softmax output of the neural network.
    y_pred = tf.nn.softmax(logits=logits)
    
    # Classification output of the neural network.
    y_pred_cls = tf.argmax(y_pred, axis=1)
   
    if mode == tf.estimator.ModeKeys.PREDICT:
        # If the estimator is supposed to be in prediction-mode
        # then use the predicted class-number that is output by
        # the neural network. Optimization etc. is not needed.
        spec = tf.estimator.EstimatorSpec(mode=mode,
                                          predictions=y_pred_cls)
    else:
        # Otherwise the estimator is supposed to be in either
        # training or evaluation-mode. Note that the loss-function
        # is also required in Evaluation mode.
        
        # Define the loss-function to be optimized, by first
        # calculating the cross-entropy between the output of
        # the neural network and the true labels for the input data.
        # This gives the cross-entropy for each image in the batch.
        cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels,
                                                                       logits=logits)

        # Reduce the cross-entropy batch-tensor to a single number
        # which can be used in optimization of the neural network.
        loss = tf.reduce_mean(cross_entropy)
        # Define the optimizer for improving the neural network.
        optimizer = tf.train.AdamOptimizer(learning_rate=1e-4)

        # Get the TensorFlow op for doing a single optimization step.
        train_op = optimizer.minimize(
            loss=loss, global_step=tf.train.get_global_step())
        # Define the evaluation metrics,
        # in this case the classification accuracy.
        accuracy = tf.metrics.accuracy(labels, y_pred_cls)
#         accuracy = tf.metrics.mean_per_class_accuracy(labels, y_pred_cls,num_classes=4)
        
        metrics = \
        {
            "Accuracy": accuracy

        }
        if mode == tf.estimator.ModeKeys.TRAIN:
            tf.summary.scalar('Train_accuracy', accuracy[1])
  
        # Wrap all of this in an EstimatorSpec.
        spec = tf.estimator.EstimatorSpec(
            mode=mode,
            loss=loss,
            train_op=train_op,
            eval_metric_ops=metrics)
        
    return spec
dnn_classifier = tf.estimator.Estimator(model_fn=dnn_model_fn,params=params,
                                        model_dir=os.path.join(model_dir, 'dnn'),)
train_and_eval(dnn_classifier,my_num_epochs,my_steps)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_keep_checkpoint_max': 5, '_device_fn': None, '_task_id': 0, '_master': '', '_train_distribute': None, '_service': None, '_num_worker_replicas': 1, '_eval_distribute': None, '_task_type': 'worker', '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_evaluation_master': '', '_save_summary_steps': 100, '_model_dir': '/home/jiv/TF/tmpy4i6ym3p/dnn', '_save_checkpoints_steps': None, '_experimental_distribute': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f9318080eb8>, '_tf_random_seed': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_num_ps_replicas': 0, '_protocol': None, '_keep_checkpoint_every_n_hours': 10000, '_global_id_in_cluster': 0, '_is_chief': True}
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
To construct input pipelines, 

INFO:tensorflow:Loss for final step: 0.06621597.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-05-29T10:43:10Z
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /home/jiv/TF/tmpy4i6ym3p/dnn/model.ckpt-3730
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-05-29-10:43:10
INFO:tensorflow:Saving dict for global step 3730: Accuracy = 0.7594221, global_step = 3730, loss = 0.7721328
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 3730: /home/jiv/TF/tmpy4i6ym3p/dnn/model.ckpt-3730
Eval Accuracy is 
{'loss': 0.7721328, 'Accuracy': 0.7594221, 'global_step': 3730}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /home/

### 2) CNN based custom estimator

In [20]:

params = {'embedding_initializer': my_initializer}
def cnn_model_fn(features, labels, mode, params):
       
    input_layer = tf.contrib.layers.embed_sequence(ids=features["x"], embed_dim=size,vocab_size=VOC_size,
                                                   initializer=params['embedding_initializer'],
                                                   trainable=True)
    #######################################################################################
    training = mode == tf.estimator.ModeKeys.TRAIN
    dropout_emb = tf.layers.dropout(inputs=input_layer, rate=0.2, training=training)
    print('dropout_emb layer')
    print(input_layer.shape.dims)
    conv = tf.layers.conv1d( inputs=dropout_emb,filters=150,kernel_size=25,padding="same", activation=tf.nn.relu)
    print('conv layer')
    print(conv.shape.dims)
    pool = tf.layers.max_pooling1d(inputs = conv, pool_size = 2, strides = 1)
    print('pool layer')
    print(pool.shape.dims)
    flat = tf.contrib.layers.flatten(pool)
    print('flat layer')
    print(flat.shape.dims)
#     hidden = tf.layers.dense(inputs=flat, units = 40, activation=tf.nn.relu)
#     dropout_hidden = tf.layers.dropout(inputs=hidden,rate=0.2,training=training)
    
    logits = tf.layers.dense(inputs=flat , units=4)
    ######################################################################################
    # Softmax output of the neural network.
    y_pred = tf.nn.softmax(logits=logits)
    
    # Classification output of the neural network.
    y_pred_cls = tf.argmax(y_pred, axis=1)
   
    if mode == tf.estimator.ModeKeys.PREDICT:
        # If the estimator is supposed to be in prediction-mode
        # then use the predicted class-number that is output by
        # the neural network. Optimization etc. is not needed.
        spec = tf.estimator.EstimatorSpec(mode=mode,
                                          predictions=y_pred_cls)
    else:
        # Otherwise the estimator is supposed to be in either
        # training or evaluation-mode. Note that the loss-function
        # is also required in Evaluation mode.
        
        # Define the loss-function to be optimized, by first
        # calculating the cross-entropy between the output of
        # the neural network and the true labels for the input data.
        # This gives the cross-entropy for each image in the batch.
        cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels,
                                                                       logits=logits)

        # Reduce the cross-entropy batch-tensor to a single number
        # which can be used in optimization of the neural network.
        loss = tf.reduce_mean(cross_entropy)
        # Define the optimizer for improving the neural network.
        optimizer = tf.train.AdamOptimizer(learning_rate=1e-4)

        # Get the TensorFlow op for doing a single optimization step.
        train_op = optimizer.minimize(
            loss=loss, global_step=tf.train.get_global_step())
        # Define the evaluation metrics,
        # in this case the classification accuracy.
        accuracy = tf.metrics.accuracy(labels, y_pred_cls)
#         accuracy = tf.metrics.mean_per_class_accuracy(labels, y_pred_cls,num_classes=4)
        
        metrics = \
        {
            "Accuracy": accuracy

        }
        if mode == tf.estimator.ModeKeys.TRAIN:
            tf.summary.scalar('Train_accuracy', accuracy[1])
  
        # Wrap all of this in an EstimatorSpec.
        spec = tf.estimator.EstimatorSpec(
            mode=mode,
            loss=loss,
            train_op=train_op,
            eval_metric_ops=metrics)
        
    return spec

cnn_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn, params=params,
                                        model_dir=os.path.join(model_dir, 'cnn'))
train_and_eval(cnn_classifier,my_num_epochs,my_steps)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_keep_checkpoint_max': 5, '_device_fn': None, '_task_id': 0, '_master': '', '_train_distribute': None, '_service': None, '_num_worker_replicas': 1, '_eval_distribute': None, '_task_type': 'worker', '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_evaluation_master': '', '_save_summary_steps': 100, '_model_dir': '/home/jiv/TF/tmpy4i6ym3p/cnn', '_save_checkpoints_steps': None, '_experimental_distribute': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f933d5f3c18>, '_tf_random_seed': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_num_ps_replicas': 0, '_protocol': None, '_keep_checkpoint_every_n_hours': 10000, '_global_id_in_cluster': 0, '_is_chief': True}
INFO:tensorflow:Calling model_fn.
dropout_emb layer
[Dimension(None), Dimension(2), Dimension(350)]
Instructions for updating:
Us

INFO:tensorflow:Restoring parameters from /home/jiv/TF/tmpy4i6ym3p/cnn/model.ckpt-3730
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-05-29-10:43:34
INFO:tensorflow:Saving dict for global step 3730: Accuracy = 0.76842105, global_step = 3730, loss = 0.70571625
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 3730: /home/jiv/TF/tmpy4i6ym3p/cnn/model.ckpt-3730
Test Accuracy is 
{'loss': 0.70571625, 'Accuracy': 0.76842105, 'global_step': 3730}
INFO:tensorflow:Calling model_fn.
dropout_emb layer
[Dimension(None), Dimension(2), Dimension(350)]
conv layer
[Dimension(None), Dimension(2), Dimension(150)]
pool layer
[Dimension(None), Dimension(1), Dimension(150)]
flat layer
[Dimension(None), Dimension(150)]
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /home/jiv/TF/tmpy4i6ym3p/cnn/model.ckpt-3730
INFO:tensorflow:Running local_init

### 3) LSTM based custom estimator

In [21]:
params = {'embedding_initializer': my_initializer}
def LSTM_model_fn(features, labels, mode, params):
    input_layer = tf.contrib.layers.embed_sequence(ids=features["x"], embed_dim=size,vocab_size=VOC_size,
                                              initializer=params['embedding_initializer'],
                                              trainable=True)
    #################################################################
    training = mode == tf.estimator.ModeKeys.TRAIN
    dropout_emb = tf.layers.dropout(inputs=input_layer, rate=0.2, training=training)
    print('input layer')
    print(input_layer.shape.dims)
    # create an LSTM cell of size 100
    lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(100)
    
    # create the complete LSTM
    _, final_states = tf.nn.dynamic_rnn(lstm_cell, dropout_emb, dtype=tf.float32)
    

    # get the final hidden states of dimensionality [batch_size x sentence_size]
    outputs = final_states.h
    print('LSTM layer')
    print(outputs.shape.dims)
#     h1 = tf.layers.dense(inputs=outputs, units=50)
#     h2= tf.layers.dense(inputs=h1, units=50)

    logits = tf.layers.dense(inputs=outputs, units=4)
        # Softmax output of the neural network.
    y_pred = tf.nn.softmax(logits=logits)
    
    # Classification output of the neural network.
    y_pred_cls = tf.argmax(y_pred, axis=1)
    #################################################################


    # Softmax output of the neural network.
    y_pred = tf.nn.softmax(logits=logits)
    
    # Classification output of the neural network.
    y_pred_cls = tf.argmax(y_pred, axis=1)
   
    if mode == tf.estimator.ModeKeys.PREDICT:
        # If the estimator is supposed to be in prediction-mode
        # then use the predicted class-number that is output by
        # the neural network. Optimization etc. is not needed.
        spec = tf.estimator.EstimatorSpec(mode=mode,
                                          predictions=y_pred_cls)
    else:
        # Otherwise the estimator is supposed to be in either
        # training or evaluation-mode. Note that the loss-function
        # is also required in Evaluation mode.
        
        # Define the loss-function to be optimized, by first
        # calculating the cross-entropy between the output of
        # the neural network and the true labels for the input data.
        # This gives the cross-entropy for each image in the batch.
        cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels,
                                                                       logits=logits)

        # Reduce the cross-entropy batch-tensor to a single number
        # which can be used in optimization of the neural network.
        loss = tf.reduce_mean(cross_entropy)

        # Define the optimizer for improving the neural network.
        optimizer = tf.train.AdamOptimizer(learning_rate=1e-4)

        # Get the TensorFlow op for doing a single optimization step.
        train_op = optimizer.minimize(
            loss=loss, global_step=tf.train.get_global_step())

        # Define the evaluation metrics,
        # in this case the classification accuracy.
        accuracy=tf.metrics.accuracy(labels, y_pred_cls)
        metrics = \
        {
            "accuracy": accuracy
    
        }
        if mode == tf.estimator.ModeKeys.TRAIN:
            tf.summary.scalar('Train_accuracy', accuracy[1])

        # Wrap all of this in an EstimatorSpec.
        spec = tf.estimator.EstimatorSpec(
            mode=mode,
            loss=loss,
            train_op=train_op,
            eval_metric_ops=metrics)
        
    return spec

LSTM_classifier = tf.estimator.Estimator(model_fn=LSTM_model_fn,params=params,
                                          model_dir=os.path.join(model_dir, 'LSTM'))
train_and_eval(LSTM_classifier,my_num_epochs,my_steps)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_keep_checkpoint_max': 5, '_device_fn': None, '_task_id': 0, '_master': '', '_train_distribute': None, '_service': None, '_num_worker_replicas': 1, '_eval_distribute': None, '_task_type': 'worker', '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_evaluation_master': '', '_save_summary_steps': 100, '_model_dir': '/home/jiv/TF/tmpy4i6ym3p/LSTM', '_save_checkpoints_steps': None, '_experimental_distribute': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f931a208f98>, '_tf_random_seed': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_num_ps_replicas': 0, '_protocol': None, '_keep_checkpoint_every_n_hours': 10000, '_global_id_in_cluster': 0, '_is_chief': True}
INFO:tensorflow:Calling model_fn.
input layer
[Dimension(None), Dimension(2), Dimension(350)]
Instructions for updating:
This cl

Test Accuracy is 
{'loss': 0.6159573, 'accuracy': 0.77293235, 'global_step': 3730}
INFO:tensorflow:Calling model_fn.
input layer
[Dimension(None), Dimension(2), Dimension(350)]
LSTM layer
[Dimension(None), Dimension(100)]
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /home/jiv/TF/tmpy4i6ym3p/LSTM/model.ckpt-3730
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
              precision    recall  f1-score   support

           0       0.74      0.59      0.66        54
           1       0.82      0.82      0.82       860
           2       0.67      0.63      0.65       228
           3       0.76      0.77      0.77       853

   micro avg       0.77      0.77      0.77      1995
   macro avg       0.75      0.70      0.72      1995
weighted avg       0.77      0.77      0.77      1995

[[ 32   5   1  16]
 [  1 705  19 135]
 [  0  23 144  61]
 [ 10 130  52 661]]


In [22]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15041255278225040292
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 9351469408927553617
physical_device_desc: "device: XLA_GPU device"
, name: "/device:XLA_GPU:1"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 13249576032491219311
physical_device_desc: "device: XLA_GPU device"
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 17099725316326666415
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 10813738189
locality {
  bus_id: 1
  links {
  }
}
incarnation: 11998964085657888121
physical_device_desc: "device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:17:00.0, compute capability: 7.5"
, name: "/device:GPU:1"
device_type: "GPU"
memory_limit: 10811247821
locality {
  bus_id: 1
  links {
  }
}
incarnation: 

In [23]:
# filepath = '../LSTM_error' 
# raw=[]  
# with open(filepatah) as fp:  
#     line = fp.readline()
#     cnt = 0
#     while line:
#         tmp=[]
#         for word in line.split():
#             tmp.append(word)
#         raw.append(tmp)
#         line = fp.readline()
#         cnt += 1
# print('No of lines in corpus are '+ str(cnt))


In [24]:
l=0
for i in range(0,len(raw)):
    if raw[i][2]=='T' and raw[i][3]=='A':
        print(raw[i])
        l=l+1
l

NameError: name 'raw' is not defined

In [ ]:
print('work')

In [ ]:
# input_data=pd.DataFrame(raw)
# # input_data.head()
# data=[input_data.iloc[:,2],input_data.iloc[:,3]] 
# len(data)

In [ ]:

# input_data

In [ ]:
# # import alluvial
# import matplotlib.pyplot as plt
# import matplotlib.cm
# import numpy as np

# # Generating the input_data:
# seed=7
# np.random.seed(seed)
# def rand_letter(num): return chr(ord('A')+int(num*np.random.rand()))
# input_data = [[raw[i][2], raw[i][3]] for i in range(0,len(raw))]
# # input_data = [[rand_letter(15), rand_letter(5)*2] for _ in range(50)]
# # input_data=[raw[:,2],raw[:,3]]
# # Plotting:
# cmap = matplotlib.cm.get_cmap('jet')
# ax = plot(
#     input_data,  alpha=0.8, color_side=0, rand_seed=None, figsize=(7,5),
#     disp_width=False, wdisp_sep=' '*2, cmap=cmap, fontname='Monospace',
#     labels=('True class', 'predicted class'),v_gap_frac=0.09,h_gap_frac=0.05,  label_shift=4)
# ax.set_title('Misclassification flow', fontsize=1, fontname='Monospace')
# plt.show()

In [ ]:
# test_results = LSTM_classifier.predict(input_fn=my_test_fn())
# test_predictions=[]
# for pred in test_results:
#     test_predictions.append(pred)

In [ ]:
# input_data

In [ ]:

# import matplotlib.pyplot as plt
# import matplotlib.cm
# import numpy as np

# # Generating the input_data:
# seed=7
# np.random.seed(seed)
# def rand_letter(num): return chr(ord('A')+int(num*np.random.rand()))

# # input_data = [[rand_letter(15), rand_letter(5)*2] for _ in range(50)]
# input_data = [[raw[i][2], raw[i][3]] for i in range(0,len(raw))]
# # Plotting:
# cmap = matplotlib.cm.get_cmap('jet')
# ax = plot(
#     input_data,  alpha=0.8, color_side=0,rand_seed=1, figsize=(7,5),
#     disp_width=False, wdisp_sep=1*' ',v_gap_frac=0.09,h_gap_frac=0.1, cmap=None, fontname='Monospace',
#     labels=('', ''),  label_shift=2,width_in=True)
# ax.set_title('', fontsize=1, fontname='Monospace')
# plt.show()

In [ ]:
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
e = [[33,4,0,17],
     [0,707,20,133],
     [0,26,148,54],
     [8,131,44,670]]      
for row in range(4):
    e[row] /= np.sum(e[row])
df_cm = pd.DataFrame(e,columns=['A','B','D','T'])
df_cm.index=['A','B','D','T']
#plt.figure(figsize = (10,7))
sn.set(font_scale=1.25)#for label size
sn.heatmap(df_cm, annot=True,center= 1,annot_kws={"size": 16})# font size
plt.xlabel('Predicted Classes',fontsize=14)
plt.ylabel('Ground Truth Classes',fontsize=14)